In [1]:
import altair as alt
import pandas as pd
from altair import datum
from altair import data_transformers

alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [2]:
# Load data from CSV file
df = pd.read_csv('Boonsong Lekagul waterways readings.csv')

df

,id,value,location,sample date,measure
0,2221,2.00,Boonsri,11-Jan-98,Water temperature
1,2223,9.10,Boonsri,11-Jan-98,Dissolved oxygen
2,2227,0.33,Boonsri,11-Jan-98,Ammonium
3,2228,0.01,Boonsri,11-Jan-98,Nitrites
4,2229,1.47,Boonsri,11-Jan-98,Nitrates
...,...,...,...,...,...
136819,3295800,5.20,Chai,27-Dec-16,Water temperature
136820,3295802,5.20,Chai,28-Dec-16,Water temperature
136821,3295804,5.00,Chai,29-Dec-16,Water temperature
136822,3295806,4.60,Chai,30-Dec-16,Water temperature


In [3]:
df.isna().sum()

id             0
value          0
location       0
sample date    0
measure        0
dtype: int64

In [4]:
df['location'].unique()

array(['Boonsri', 'Kannika', 'Chai', 'Kohsoom', 'Somchair', 'Sakda',
       'Busarakhan', 'Tansanee', 'Achara', 'Decha'], dtype=object)

In [5]:
df['measure'].unique()

array(['Water temperature', 'Dissolved oxygen', 'Ammonium', 'Nitrites',
       'Nitrates', 'Orthophosphate-phosphorus', 'Total phosphorus',
       'Sodium', 'Potassium', 'Calcium', 'Magnesium', 'Chlorides',
       'Sulphates', 'Iron', 'Manganese', 'Zinc', 'Copper', 'Chromium',
       'Lead', 'Cadmium', 'Mercury', 'Nickel', 'Arsenic',
       'Biochemical Oxygen', 'Chemical Oxygen Demand (Cr)',
       'Chemical Oxygen Demand (Mn)', 'AOX', 'Atrazine', 'Cesium',
       'Macrozoobenthos', 'Total coliforms', 'Fecal coliforms', 'p,p-DDT',
       'gamma-Hexachlorocyclohexane', 'Bicarbonates',
       'Anionic active surfactants', 'Total extractable matter',
       'Fecal streptococci ', 'Petroleum hydrocarbons', 'PAHs',
       'Benzo(a)pyrene', 'Benzo(g,h,i)perylene', 'Benzo(b)fluoranthene',
       'Benzo(k)fluoranthene', 'Fluoranthene', 'Indeno(1,2,3-c,d)pyrene',
       'PCB 28', 'PCB 52', 'PCB 101', 'PCB 138', 'PCB 153', 'PCB 180',
       'Silica (SiO2)', 'Oxygen saturation', 'Total hardness'

# Finding 1

<font size="4"><b>Insight 1: Overall trend of contamination levels over time at highest contaminated location</b></font>

In [6]:
# Finding the location with the highest contamination
max_value = df['value'].max()
df_max_contamination = df[df['value'] == max_value]
high_contaminated_location = df_max_contamination['location'].values[0]

# Convert the 'sample date' column to datetime format
df['sample date'] = pd.to_datetime(df['sample date'])

# Create a bar chart to show the contamination levels by location

chart1 = alt.Chart(df).mark_bar().encode(
    x=alt.X('location:N', title='Location'), 
    y=alt.Y('value:Q', title='Contamination Level'), 
    color='location:N',
    tooltip=['location', 'value']
).properties(
    width=700,
    title='Overall trend of contamination levels over time at highest contaminated location'
)

chart1

alt.Chart(...)

<font size="4"><b>Insight 2: Comparison of contamination levels between different chemicals to find highest value for chemical in Kohsoom<b><font>

In [7]:
# Filter data for Kohsoom location
new_df = df[df['location'] == 'Kohsoom'].copy()

# Create a stacked area chart with gradient color scale
chart2 = alt.Chart(new_df).mark_bar().encode(
    x=alt.X('year(sample date):N', title='Year'),
    y=alt.Y('value:Q', title='Contamination Level')
).properties(
    title='Comparison of contamination levels across chemicals in Kohsoom'
)

chart2

alt.Chart(...)

<font size="4"><b>Insight 3: Seasonal patterns for Iron at Kohsoom in 2003</b></font>

In [8]:
# Filter the data for the specific location and measure
location = 'Kohsoom'
measure = 'Iron'

new_df_filtered = new_df[(new_df['location'] == location) & (new_df['measure'] == measure) & (new_df['sample date'].dt.year == 2003)].copy()

# Create an area chart with interactive selection
chart3 = alt.Chart(new_df_filtered).mark_area(
    line={'color': 'white'}
).encode(
    x=alt.X('month(sample date):T', title='Months'),
    y=alt.Y('value:Q', title='Contamination Level'),
    color='measure:N',
    tooltip=['measure', 'value']
).properties(
    title='Seasonal patterns for Iron at Kohsoom in 2003'
).interactive()

chart3

alt.Chart(...)

# Dashboard 1

In [9]:
dashboard = alt.vconcat(chart1, alt.hconcat(chart2, chart3))
dashboard

alt.VConcatChart(...)

# Finding 2

<font size="4"><b>Insight 1: Comparison of chemical contamination levels across different sensor sites at the same point in time</b></font>

In [10]:
df_year = df.copy()
df_year['Year'] = df_year['sample date'].dt.year

# Calculate mean contamination level for each year
yearly_mean = df_year.groupby('Year')['value'].mean().reset_index()

# Find year with lowest mean contamination level
min_year = yearly_mean.loc[yearly_mean['value'].idxmin()]['Year']

# Create a selection for the year
year_selection = alt.selection_single(
    name='Year',
    fields=['Year'],
    init={'Year': min_year},
    bind=alt.binding_select(options=yearly_mean['Year'].tolist())
)

# Create the chart
chart1 = alt.Chart(df_year).mark_area(opacity=0.7).encode(
    x=alt.X('month(sample date):T', title='Month'),
    y=alt.Y('mean(value):Q', title='Mean Contamination Level'),
    color=alt.Color('Year:N', title='Year', scale=alt.Scale(scheme='reds')),
    tooltip=['Year', 'mean(value)', 'month(sample date)'],
    opacity=alt.condition(year_selection, alt.value(1), alt.value(0.2))
).add_selection(year_selection)

# Show the chart
chart1

alt.Chart(...)

<font size="4"><b>Insight 2: Highest value of each measure in 2000</b></font>

In [11]:
# Filter data for measures in year 2000
data_2000 = df_year[df_year['Year'] == 2000]

# Get the highest value for each measure at each sample date
highest_values = data_2000.groupby(['measure', 'sample date']).agg({'value': 'max', 'location': 'first'}).reset_index()

# Create the scatter plot
chart2 = alt.Chart(highest_values).mark_point().encode(
    x='sample date:T',
    y='value:Q',
    color='measure:N',
    tooltip=['location', 'measure', 'value', 'sample date']
).properties(title='Highest values of each measure in 2000')

# Show the chart
chart2

alt.Chart(...)

<font size="4"><b>Insight 3: Minimum Bicarbonate values for location in 2000</b></font>

In [12]:
# Filter data for bicarbonates in year 2000
bicarb_data = df_year[(df_year['measure'] == 'Bicarbonates') & (df_year['Year'] == 2000)]

# Get the lowest bicarbonate value for each location
lowest_bicarb = bicarb_data.groupby('location')['value'].min().reset_index()

# Create the bar chart
chart3 = alt.Chart(lowest_bicarb).mark_bar().encode(
    x='location:N',
    y='value:Q',
    tooltip=['location', 'value']
).properties(width=300, title='Minimum bicarbonate values for each location in 2000')

# Show the chart
chart3

alt.Chart(...)

In [13]:
import altair as alt
import pandas as pd

# Convert the sample_date column to year format
new_df_format = df.copy()
new_df_format['year'] = pd.to_datetime(new_df_format['sample date'], format='%d-%b-%y').dt.year
new_data_filtered2 = new_df_format[(new_df_format['year'] >= 1998) & (new_df_format['year'] <= 2003)]

# Compute the average value of each measure across all locations
avg_data = new_data_filtered2.groupby(['measure', 'location', 'year'])['value'].mean().reset_index()

# Create the bar chart
chart4 = alt.Chart(avg_data).mark_bar().encode(
    x=alt.X('location', sort=alt.EncodingSortField(field='value', op='mean', order='descending')),
    y='value',
    tooltip=['measure', 'location', 'year', 'value'],
    color='measure'
).properties(
    title="Average Chemical Contamination Levels Across All Locations (1998-2003)",
    width=300
)

chart4

alt.Chart(...)

# Dashboard 2

In [14]:
row1 = alt.hconcat(chart1, chart2)
row2 = alt.hconcat(chart3, chart4)
dashboard = alt.vconcat(row1, row2)
dashboard

alt.VConcatChart(...)

# Finding 3

<font size="4"><b>Insight 1: Understanding Frequency Distribution of chemicals</b></font>

In [15]:
alt.Chart(df).mark_bar().encode(
    x=alt.X('measure:N', sort='-y'),
    y='count(measure):Q'
)

alt.Chart(...)

<font size="4"><b>Insight 2: Understanding Frequency Distribution of chemicals having non zero values</b></font>

In [16]:
chart_1 = alt.Chart(df).mark_bar().encode(
    x=alt.X('measure:N', sort='-y'),
    y='count(measure):Q'
).transform_filter(
     datum.value != 0
)

chart_1

alt.Chart(...)

<font size="4"><b>Insight 3: Understanding the rate of dumpings of Nitrates and Nitrites across the years</b></font>

In [17]:
chart_2 = alt.Chart(df).mark_bar().encode(
    x="sample date:T",
    y="count(measure):Q",
    color='measure',
    tooltip=['location', 'value', 'sample date']
).transform_filter(
     alt.FieldOneOfPredicate(field='measure', oneOf=['Nitrites', 'Nitrates'])
).properties(
    width=700
)

chart_2

alt.Chart(...)

In [18]:
df_2008 = df[df['sample date'].dt.year == 2008]

df_2008

,id,value,location,sample date,measure
68396,902166,9.08,Chai,2008-01-04,Silica (SiO2)
68397,902168,0.25,Chai,2008-01-04,Tetrachloromethane
68398,902171,16.00,Chai,2008-01-04,Total coliforms
68399,902172,5.40,Chai,2008-01-04,Fecal coliforms
68400,902173,2.40,Chai,2008-01-04,Fecal streptococci
...,...,...,...,...,...
76764,902925,0.10,Chai,2008-12-21,Sulfides
76765,904059,6.11,Chai,2008-12-21,Silica (SiO2)
76766,904060,0.10,Chai,2008-12-21,Sulfides
76767,905182,6.10,Chai,2008-12-21,Silica (SiO2)


<font size="4"><b>Insight 4: Understanding the rate of dumpings of Nitrates and Nitrites across the locations in the year 2008</b></font>

In [19]:
chart_3 = alt.Chart(df_2008).mark_point(size=60).encode(
    x="location:N",
    y="count(measure):Q",
    color='measure',
    tooltip=['measure', 'value']
).transform_filter(
     alt.FieldOneOfPredicate(field='measure', oneOf=['Nitrites', 'Nitrates']),
).properties(
    width=400
)

chart_3

alt.Chart(...)

In [20]:
df_2012 = df[df['sample date'].dt.year == 2012]

df_2012

,id,value,location,sample date,measure
101851,1977123,0.0000,Tansanee,2012-01-07,Water temperature
101852,1977125,10.7300,Tansanee,2012-01-07,Dissolved oxygen
101853,1977126,75.7800,Tansanee,2012-01-07,Oxygen saturation
101854,1977128,240.3000,Tansanee,2012-01-07,Bicarbonates
101855,1977130,715.0000,Tansanee,2012-01-07,Total dissolved salts
...,...,...,...,...,...
107644,1967252,1.7515,Busarakhan,2012-12-30,Nitrates
107645,1967253,2.2700,Busarakhan,2012-12-30,Total nitrogen
107646,1967254,0.0329,Busarakhan,2012-12-30,Orthophosphate-phosphorus
107647,1967255,0.0610,Busarakhan,2012-12-30,Total phosphorus


<font size="4"><b>Insight 4: Understanding the rate of dumpings of Nitrates and Nitrites across the locations in the year 2012</b></font>

In [21]:
chart_4 = alt.Chart(df_2012).mark_point(size=60).encode(
    x="location:N",
    y="count(measure):Q",
    color='measure',
    tooltip=['measure', 'value']
).transform_filter(
     alt.FieldOneOfPredicate(field='measure', oneOf=['Nitrites', 'Nitrates']),
).properties(
    width=400
)

chart_4

alt.Chart(...)

# Dashboard 3

In [22]:
row_1=alt.vconcat(chart_1, chart_2)
row_2=alt.hconcat(chart_3, chart_4)
dashboard = alt.vconcat(row_1,row_2)
dashboard

alt.VConcatChart(...)

In [25]:
alt.Chart(df).mark_bar().encode(
    x='year(sample date):T',
    y='count(distinct(measure)):Q',
    color='measure:N'
).properties(
    width=500
)

alt.Chart(...)